Here I'll try to complete task about automatization of the first trading strategy.
First of all, I'll make trading algorithm.

Trading Algorithm:
1. Getting all opened trades and count it (oped)
2. If oped less than maxTr (maxTr will define from theory in future) then creating array of symbols that are trading now (arrSymbol) else wait an hour and start from the begining (step 1)
3. Find pairs (followSymbol, array sorting by 3.2,3.4), that suit the following conditions:  
    3.1 Trends of H4 and D1 period are unidirectional  
    3.2 Trend of H4 has estimated strength (ength, that will be estimated later)  
    3.3 There is no signal of trend's change  
    3.4 Trade volume has estimated value (vol, estimates later)  
    3.5 Pair is not in arrSymbol array  
    3.6 Pair price is lower then up trend line  
    3.7 Pair price is higher then down trend line  
    3.8 Previous pair trades ended more than 24 hours ago  
4. If followSymbol is empty start again step 3
5. Follow prices of pairs from followSybol array, to fetch the moment, when price crosses the trend line
6. When the moment fetched, open long or short position
7. When position is opened (anyone), start all over again (step 1)

In [1]:
#List of all currency pairs that will be processed by robot
currencyList = ['AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'CADCHF', 'CADJPY', 'CHFJPY', 'EURAUD', 'EURCAD',
                'EURCHF', 'EURGBP', 'EURJPY', 'EURNZD', 'EURUSD', 'GBPCHF', 'GBPJPY', 'GBPUSD', 'NZDJPY', 'NZDUSD',
                'USDCAD', 'USDCHF', 'USDJPY']
timeframeList = ['1', '5', '15', '30', '60', '240', '1440', '10080']

In [2]:
from_test_file = "{'_action': 'OPEN_TRADES', '_trades': {689109449: {'_magic': 123456, '_symbol': 'AUDCAD', '_lots': 0.01000000, '_type': 0, '_open_price': 0.98220000, '_open_time': '2021.01.05 14:42:54', '_SL': 0.97820000, '_TP': 0.99020000, '_pnl': -0.31000000, '_comment': 'zmq_connector-test33'}, 689070055: {'_magic': 123456, '_symbol': 'EURUSD', '_lots': 0.01000000, '_type': 0, '_open_price': 1.22846000, '_open_time': '2021.01.05 13:39:24', '_SL': 1.22446000, '_TP': 1.23646000, '_pnl': -1.78000000, '_comment': 'zmq_connector-test32'}}}"

In [3]:
arrSymbol = [ok for ok in currencyList if ok in from_test_file]
arrSymbol

['AUDCAD', 'EURUSD']

In [4]:
#Don't know how to import function from another file, so function copied from CSVToTrend.ipynb
def trend_dir(quotes):
    import pandas as pd
    import matplotlib.dates as mdates
    import numpy as np
    _quotes = quotes
    y = np.array(_quotes[['Open', 'High', 'Low', 'Close']]).ravel()
    x = np.array(_quotes.index).ravel()
    x = np.array([(lambda d: mdates.date2num(d))(d) for d in x for _ in (0,1,2,3)])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    if m < 0:
        _dir = 1
    elif m > 0:
        _dir = 2
    else:
        _dir = 'flat' #have to organize check for sideway trend somehow
    return _dir

In [5]:
#Used from CSVToTrend.ipynb
import pandas as pd
col_names=['Date', 'Time','Open','High','Low','Close','Tick_Vol']
df = pd.read_csv ('~/anaconda3/envs/FR/EURUSD1440.csv',names=col_names)
df['Date'] = df['Date']+' '+df['Time']
df['DateTime']=pd.to_datetime(df.pop('Date'))
D1 = df[-11:].set_index('DateTime')
del D1['Time']
df = pd.read_csv ('~/anaconda3/envs/FR/EURUSD240.csv',names=col_names)
df['Date'] = df['Date']+' '+df['Time']
df['DateTime']=pd.to_datetime(df.pop('Date'))
H4 = df[-31:].set_index('DateTime')
del H4['Time']

In [43]:
#Function creates dataframe with qoutes with specified currency pair and specified timeframe
def get_df(path, cur_pair, tf):
    import os
    import pandas as pd
    for name in os.listdir(path):
        cur_path = os.path.join(path, name)
        if os.path.isfile(cur_path) and cur_pair in name and tf in name:
            col_names=['Date', 'Time','Open','High','Low','Close','Tick_Vol']
            df = pd.read_csv (cur_path, names=col_names)
            df['Date'] = df['Date']+' '+df['Time']
            df['DateTime']=pd.to_datetime(df.pop('Date'))
            my_df = df[-40:].set_index('DateTime')
            del my_df['Time']
            return my_df

In [7]:
#For "3.1 Trends of H4 and D1 period are unidirectional"
import os
trend_dir(get_df(os.pardir, 'EURUSD', '1440')) == trend_dir(get_df(os.pardir, 'EURUSD', '240'))

True

In [8]:
#Function gets m that is calculated the same way as it was calculated in treng_dir function
def get_ength(quotes):
    import pandas as pd
    import matplotlib.dates as mdates
    import numpy as np
    _quotes = quotes
    y = np.array(_quotes[['Open', 'High', 'Low', 'Close']]).ravel()
    x = np.array(_quotes.index).ravel()
    x = np.array([(lambda d: mdates.date2num(d))(d) for d in x for _ in (0,1,2,3)])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m

In [9]:
get_ength(get_df(os.pardir, 'EURUSD', '1440'))

-0.004381151746645721

In [10]:
import os
from numpy import median, mean
m_in_EURUSD = []
for cur_tf in timeframeList:
    cur_df = get_df(os.getcwd(), 'EURUSD', cur_tf)
    if  cur_df is not None:
        m_in_EURUSD.append(abs(get_ength(cur_df)))
print(m_in_EURUSD)
print(median(m_in_EURUSD), min(m_in_EURUSD), max(m_in_EURUSD), mean(m_in_EURUSD))

[0.00015139271255627622, 0.030076363611040473, 0.009798545452040851, 0.006116727272234097, 0.0032852727286320892, 0.0012104659950712049, 0.00015139271255627622, 0.00038988961038965135]
0.002247869361851647 0.00015139271255627622 0.030076363611040473 0.006397506261815115


In [11]:
#3.2 Trend of H4 has estimated strength
median(m_in_EURUSD) < get_ength(get_df(os.pardir, 'EURUSD', '240'))

False

In [12]:
#Calculating RSI to know that "There is no signal of trend's change"
import numpy as np
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)


#sample data from StockCharts
data = pd.Series( [ 44.34, 44.09, 44.15, 43.61,
                    44.33, 44.83, 45.10, 45.42,
                    45.84, 46.08, 45.89, 46.03,
                    45.61, 46.28, 46.28, 46.00,
                    46.03, 46.41, 46.22, 45.64 ] )
print (RSI( data, len(data)-1 ))

19    60.483871
dtype: float64


In [13]:
mmm = get_df(os.pardir, 'EURUSD', '1440')['Tick_Vol']
print(mmm.min(), mmm.max(), mmm.mean(), mmm.quantile(0.9))

73140 135665 106499.09090909091 118724.0


In [14]:
def check_price(price, time, m, c):
    trend_dot = time * m + c
    if (price>trend_dot)&(m<0):
        return True
    if (price<trend_dot)&(m>0):
        return True
    return False

In [23]:
def traversal_fiveless(quotes):
    import pandas as pd
    import matplotlib.dates as mdates
    import numpy as np
    y = np.array(quotes[['Open', 'High', 'Low', 'Close']]).ravel()
    x = np.array(quotes.index).ravel()
    x = np.array([(lambda d: mdates.date2num(d))(d) for d in x for _ in (0,1,2,3)])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    x_scale = pd.DataFrame(x).drop_duplicates()
    primary = check_price(quotes.Close[0], x_scale[0].iloc[0], m, c)
    traversal = 0
    quotes['date_to_num'] = x_scale[0].values
    for t in x_scale[0]:        t_prim = check_price(quotes.Close.loc[quotes['date_to_num'] == t].values[0], t, m, c)
        if primary == t_prim:
            continue
        primary = t_prim
        traversal += 1
        if traversal > 4:
            return False
    return True
        t_prim = check_price(quotes.Close.loc[quotes['date_to_num'] == t].values[0], t, m, c)
        if primary == t_prim:
            continue
        primary = t_prim
        traversal += 1
        if traversal > 4:
            return False
    return True

In [44]:
traversal_fiveless(get_df(os.pardir, 'EURUSD', '1440'))

True